In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c rsna-pneumonia-detection-challenge
!unzip rsna-pneumonia-detection-challenge.zip

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 407, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method. See setup instructions at https://github.com/Kaggle/kaggle-api/
unzip:  cannot find or open rsna-pneumonia-detection-challenge.zip, rsna-pneumonia-detection-challenge.zip.zip or rsna-pneumonia-detection-challenge.zip.ZIP.


In [ ]:
!pip install --upgrade pylibjpeg pylibjpeg-libjpeg pydicom pillow

In [ ]:
import cv2
import numpy as np
import pydicom
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pydicom'

In [ ]:
import pandas as pd

# Load the training labels CSV
train_labels = pd.read_csv('/content/stage_2_train_labels.csv')

# Filter the images with pneumonia (Target = 1)
pneumonia_images = train_labels[train_labels['Target'] == 1]['patientId'].unique()

print(f"Number of images with pneumonia: {len(pneumonia_images)}")

In [ ]:
from PIL import Image
import numpy as np
import pydicom
import os

def dicom_to_jpeg(dicom_file, jpeg_file):
    # Read the DICOM file
    dicom_data = pydicom.dcmread(dicom_file)

    # Ensure the DICOM file has pixel data
    if 'PixelData' not in dicom_data:
        print(f"No pixel data found in {dicom_file}")
        return False

    # Convert DICOM pixel data to a numpy array
    image_data = dicom_data.pixel_array

    # Normalize the image data to range [0, 255]
    image_data = (image_data / np.max(image_data)) * 255
    image_data = image_data.astype(np.uint8)

    # Convert the numpy array to a PIL image and save as JPEG
    image = Image.fromarray(image_data)
    image.save(jpeg_file)

    return True

# Set the folder paths
dicom_folder = '/content/stage_2_train_images'
jpeg_folder = '/content/converted_jpeg_images'
os.makedirs(jpeg_folder, exist_ok=True)

# Convert the first 10 images with pneumonia to JPEG
for i, patient_id in enumerate(pneumonia_images[:10]):
    dicom_file = f"{dicom_folder}/{patient_id}.dcm"
    jpeg_file = f"{jpeg_folder}/{patient_id}.jpg"

    if dicom_to_jpeg(dicom_file, jpeg_file):
        print(f"Successfully converted {dicom_file} to {jpeg_file}")
    else:
        print(f"Failed to convert {dicom_file}")

Successfully converted /content/stage_2_train_images/00436515-870c-4b36-a041-de91049b9ab4.dcm to /content/converted_jpeg_images/00436515-870c-4b36-a041-de91049b9ab4.jpg
Successfully converted /content/stage_2_train_images/00704310-78a8-4b38-8475-49f4573b2dbb.dcm to /content/converted_jpeg_images/00704310-78a8-4b38-8475-49f4573b2dbb.jpg
Successfully converted /content/stage_2_train_images/00aecb01-a116-45a2-956c-08d2fa55433f.dcm to /content/converted_jpeg_images/00aecb01-a116-45a2-956c-08d2fa55433f.jpg
Successfully converted /content/stage_2_train_images/00c0b293-48e7-4e16-ac76-9269ba535a62.dcm to /content/converted_jpeg_images/00c0b293-48e7-4e16-ac76-9269ba535a62.jpg
Successfully converted /content/stage_2_train_images/00f08de1-517e-4652-a04f-d1dc9ee48593.dcm to /content/converted_jpeg_images/00f08de1-517e-4652-a04f-d1dc9ee48593.jpg
Successfully converted /content/stage_2_train_images/0100515c-5204-4f31-98e0-f35e4b00004a.dcm to /content/converted_jpeg_images/0100515c-5204-4f31-98e0-f35

In [ ]:
import cv2
import matplotlib.pyplot as plt

def apply_clahe(image_path):
    # Read the JPEG image
    image = cv2.imread(image_path)

    if image is None:
        print(f"Error: Could not read the image at {image_path}.")
        return None, None

    # Convert the image to grayscale
    image_bw = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Initialize CLAHE
    clahe = cv2.createCLAHE(clipLimit=5)

    # Apply CLAHE
    clahe_image = clahe.apply(image_bw) + 30

    return image_bw, clahe_image

# Display the original and CLAHE-applied images side by side for the first 10 images
for i, patient_id in enumerate(pneumonia_images[:10]):
    jpeg_image_path = f"/content/converted_jpeg_images/{patient_id}.jpg"

    original_image, clahe_image = apply_clahe(jpeg_image_path)

    if clahe_image is not None:
        # Plot the original and CLAHE-applied images side by side
        fig, ax = plt.subplots(1, 2, figsize=(12, 6))
        ax[0].imshow(original_image, cmap='gray')
        ax[0].set_title(f"Original Image: {patient_id}")
        ax[0].axis('off')

        ax[1].imshow(clahe_image, cmap='gray')
        ax[1].set_title(f"CLAHE Applied: {patient_id}")
        ax[1].axis('off')

        plt.show()

NameError: name 'pneumonia_images' is not defined